# BatSimuPOP

# Simulation of population changes on bat colonies over time to assess the use of temporal sampling

# Input file:
1 Genepop file with 2 subpopulations (total number of individuals in genepop is inferior to the number of individuals in the simulation)

subpop 1: local colony that will undergo changes (additional info on age, age category and sex).

subpop 2: Rest of the population. Present to maintain gene flow over time. (additional info on sex and whether juvenile or adult)

Information on sex, age in years (for 1 subpop) or age category (juvenile-adult) is also available (file in fstat format 14 microsatellite loci + additional info)

In [1]:
from simuOpt import setOptions
setOptions(quiet=True)
import simuPOP as sim
from simuPOP.utils import importPopulation

pop = importPopulation('GENEPOP', 'Genpop_2subpops.txt')


Ther are 68 and 192 bats in two populations. No sex and age information yet.

In [2]:
# get age, agecat, sex from fstat file
%preview  pop_dump.txt --limit 500
import random

pop.addInfoFields(['age'])

# age_cat is not imported as they are defined by age 
# 0, 1, 2 as juvenile and 2-above as adult
with open('Fstat_dat(age,agecat,sex).txt') as fs:
    for idx, line in enumerate(fs):
        if idx < 10:
            # skip the first few lines
            continue
        if line.startswith('1 '):
            age, age_cat, sex = line.strip().split()[-3:]
            pop.individual(idx-18).setSex(sim.MALE if sex == '2' else sim.FEMALE )
            pop.individual(idx-18).age = int(age)
            #pop.individual(idx-18).age_cat = int(age_cat)
        elif line.startswith('2 '):
            age_cat, sex = line.strip().split()[-2:]
            pop.individual(idx-18).setSex(sim.MALE if sex == '2' else sim.FEMALE )
            # no age info
            pop.individual(idx-18).age = random.randint(0, 17)
            #pop.individual(idx-18).age_cat = int(age_cat)
sim.dump(pop, width=3, max=300, output='pop_dump.txt')     

In [24]:
pop.setVirtualSplitter(
    sim.CombinedSplitter([
        sim.SexSplitter(),
        sim.InfoSplitter(field='age', cutoff=[3, 17] ),
    ]))

for x in range(0, 4):
    print(pop.subPopName([0, x]))
    print(pop.indInfo('age', subPop=[0, x]))

Male
(2.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0, 9.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 6.0, 1.0, 1.0, 12.0, 14.0, 13.0, 12.0, 9.0, 8.0, 6.0, 15.0, 15.0, 11.0, 13.0, 13.0, 12.0, 9.0, 2.0, 8.0, 10.0, 10.0, 7.0, 7.0, 8.0, 7.0, 7.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 5.0, 5.0, 5.0, 5.0, 5.0)
Female
(2.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0, 9.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 6.0, 1.0, 1.0, 12.0, 14.0, 13.0, 12.0, 9.0, 8.0, 6.0, 15.0, 15.0, 11.0, 13.0, 13.0, 12.0, 9.0, 2.0, 8.0, 10.0, 10.0, 7.0, 7.0, 8.0, 7.0, 7.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0)
age < 3
(2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0)
3 <= age < 17
(3.0, 3.0, 3.0, 3.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0, 9.0, 6.0, 12.0, 14.0, 13.0, 12.0, 9.0, 8.0, 6.0, 15.0, 15.0, 11.0, 13.0, 13.0, 12.0, 9.0, 8.0

# Other settings:

Random mating in subpopulations

Maintain gene flow between both sub-populations using Migrator. This is sex-biased as females remain in the same colony and males disperse.

Female migration rate	0.001

Male migration rate	0.1

I think it might be better to start with the 68 females from the first population, then create an additional 62 males before the start of the simulation. This will come to a total number of 130 which is close to the effective population size I calculated for that population.

For the second population, you could create 110 females and 228 males before the start of the simulation as this would give an even sex ratio for the source population and a total of 550 individuals which is close to the whole effective population size.

In [4]:
# number of males and females?
print(pop.subPopSize([0,'Male']))
print(pop.subPopSize([0,'Female']))
print(pop.subPopSize([1,'Male']))
print(pop.subPopSize([1,'Female']))

# population 0, from 68 female to 68 female + 62 male
pop.resize([130, 338], propagate=True)
# set sex
for idx in range(68, 130):
    pop.individual(idx, 0).setSex(sim.MALE)
# add 110-37=73 male
for idx in range(192, 192+73):
    pop.individual(idx, 1).setSex(sim.MALE)
for idx in range(192+73, 338):
    pop.individual(idx, 1).setSex(sim.FEMALE) 
# number of males and females?
print(pop.subPopSize([0,'Male']))
print(pop.subPopSize([0,'Female']))
print(pop.subPopSize([1,'Male']))
print(pop.subPopSize([1,'Female']))

0
68
37
155
62
68
110
228


In [5]:
# as we are keeping subpopulation 1 as the "source" population,
# no migration from 0 to 1 is needed.

migr = sim.Migrator(
    rate=[
        [0, 0.1 ],
        [0, 0.001 ],
        [0.1, 0],
        [0.001, 0],
       ],
    mode = sim.BY_PROBABILITY,
    subPops=[(0, 'Male'), (0, 'Female'), (1, 'Male'), (1, 'Female')],
    toSubPops=[0, 1],
)

# let us test the migration
pop.addInfoFields('migrate_to')

In [9]:
pop1 = pop.clone()
pop1.evolve(
    preOps=[
        sim.Stat(popSize=True,
             subPops=[(0, 0), (0, 1), (1, 0), (1, 1)],
             vars='popSize_sp'),
        sim.PyEval('''f"before migration: m/f 0: {subPop[(0,0)]['popSize']}/{subPop[(0,1)]['popSize']}''' + \
               ''' m/f 1: {subPop[(1,0)]['popSize']} {subPop[(1,1)]['popSize']}\\n"'''),
        migr,
        sim.Stat(popSize=True,
             subPops=[(0, 0), (0, 1), (1, 0), (1, 1)],
             vars='popSize_sp'),
        sim.PyEval('''f"after migration: m/f 0: {subPop[(0,0)]['popSize']}/{subPop[(0,1)]['popSize']}''' + \
               ''' m/f 1: {subPop[(1,0)]['popSize']} {subPop[(1,1)]['popSize']}\\n"'''),
    ],
    matingScheme=sim.RandomMating(), #subPopSize=[130, 338]),
    gen=5
)

before migration: m/f 0: 62/68 m/f 1: 110 228
after migration: m/f 0: 69/69 m/f 1: 103 227
before migration: m/f 0: 65/73 m/f 1: 167 163
after migration: m/f 0: 78/73 m/f 1: 154 163
before migration: m/f 0: 80/71 m/f 1: 146 171
after migration: m/f 0: 100/71 m/f 1: 126 171
before migration: m/f 0: 76/95 m/f 1: 149 148
after migration: m/f 0: 82/96 m/f 1: 143 147
before migration: m/f 0: 100/78 m/f 1: 141 149
after migration: m/f 0: 108/78 m/f 1: 133 149


5

# Example scenario:

Simulation of 20 generations (~200 years)

The initial population size is based on Ne estimates from the Genepop file (N genotypes < Actual population size), for example:

N0subpop1=130

N0subpop2=550

Subpop 2 remains constant over time

Subpop 1 undergoes an instant population change (InstantChangeModel) at generation 5 and declines to 80.

Then at generation 12 it undergoes a linear growth at rate 0.2 to a population of 150 (LinearGrowthModel) => Need of MultiStageModel 

In [17]:
def demoModel(gen):
    if gen < 50:
        return 130, 550
    elif gen < 120:
        return 80, 550
    else:
        return min(150, 80*1.2**(gen-120)), 550

pop1 = pop.clone()
pop1.evolve(
    preOps=[
        migr,
    ],
    matingScheme=sim.RandomMating(subPopSize=demoModel),
    postOps=[
        sim.Stat(popSize=True),
        sim.PyEval(r'f"{gen} {subPopSize}\n"'),
    ],
    gen=200
)

0 [130, 550]
1 [130, 550]
2 [130, 550]
3 [130, 550]
4 [130, 550]
5 [130, 550]
6 [130, 550]
7 [130, 550]
8 [130, 550]
9 [130, 550]
10 [130, 550]
11 [130, 550]
12 [130, 550]
13 [130, 550]
14 [130, 550]
15 [130, 550]
16 [130, 550]
17 [130, 550]
18 [130, 550]
19 [130, 550]
20 [130, 550]
21 [130, 550]
22 [130, 550]
23 [130, 550]
24 [130, 550]
25 [130, 550]
26 [130, 550]
27 [130, 550]
28 [130, 550]
29 [130, 550]
30 [130, 550]
31 [130, 550]
32 [130, 550]
33 [130, 550]
34 [130, 550]
35 [130, 550]
36 [130, 550]
37 [130, 550]
38 [130, 550]
39 [130, 550]
40 [130, 550]
41 [130, 550]
42 [130, 550]
43 [130, 550]
44 [130, 550]
45 [130, 550]
46 [130, 550]
47 [130, 550]
48 [130, 550]
49 [130, 550]
50 [80, 550]
51 [80, 550]
52 [80, 550]
53 [80, 550]
54 [80, 550]
55 [80, 550]
56 [80, 550]
57 [80, 550]
58 [80, 550]
59 [80, 550]
60 [80, 550]
61 [80, 550]
62 [80, 550]
63 [80, 550]
64 [80, 550]
65 [80, 550]
66 [80, 550]
67 [80, 550]
68 [80, 550]
69 [80, 550]
70 [80, 550]
71 [80, 550]
72 [80, 550]
73 [80, 550

200

# Life cycle of species to consider for overlapping generations:
Maxlifespan ~ 17 years old

minMatingAge ~ 2 years old

maxMatingAge ~ 17 years old

Number of young per year ~ 1

1 generation ~ 5 years

In [38]:
def demoModel(gen, pop):
    if gen < 50:
        sz = 130, 550
    elif gen < 120:
        sz = 80, 550
    else:
        sz = min(150, 80*1.2**(gen-120)), 550
    print(f"{gen} young/adult sp0 : {pop.subPopSize([0, 'age < 3'])} / {pop.subPopSize([0, '3 <= age < 17'])}" +
          f" sp1 : {pop.subPopSize([1, 'age < 3'])} / {pop.subPopSize([1, '3 <= age < 17'])}")
    return sz

    
    

pop1 = pop.clone()
pop1.evolve(
    preOps=[
        migr,
        sim.InfoExec('age += 1'),
    ],
    matingScheme=sim.HeteroMating([
        # only adult individuals with age >=3 will mate and produce
        # offspring. The age of offspring will be zero.
        sim.RandomMating(ops=[
            sim.MendelianGenoTransmitter()],
            subPops=[(sim.ALL_AVAIL,'3 <= age < 17')],
            weight=-0.1),
        # individuals with age < 17 will be kept, but might be removed due to
        # population size decline
        sim.CloneMating(subPops=[(sim.ALL_AVAIL, 'age < 3'), (sim.ALL_AVAIL, '3 <= age < 17')]),
        ],
        subPopSize=demoModel),
    postOps=[
        sim.Stat(popSize=True),
        sim.PyEval(r'f"{gen} {subPopSize}\n"'),
        sim.utils.Exporter(format='GENEPOP', step=10, output='!f"{gen}.pop"', gui='batch')
    ],
    gen=200
)

0 young/adult sp0 : 5 / 130 sp1 : 35 / 253
0 [130, 550]
1 young/adult sp0 : 18 / 123 sp1 : 54 / 423
1 [130, 550]
2 young/adult sp0 : 27 / 115 sp1 : 65 / 436
2 [130, 550]
3 young/adult sp0 : 26 / 110 sp1 : 81 / 430
3 [130, 550]
4 young/adult sp0 : 24 / 106 sp1 : 83 / 419
4 [130, 550]
5 young/adult sp0 : 25 / 111 sp1 : 80 / 428
5 [130, 550]
6 young/adult sp0 : 22 / 109 sp1 : 79 / 435
6 [130, 550]
7 young/adult sp0 : 24 / 119 sp1 : 80 / 444
7 [130, 550]
8 young/adult sp0 : 24 / 108 sp1 : 81 / 443
8 [130, 550]
9 young/adult sp0 : 22 / 110 sp1 : 84 / 433
9 [130, 550]
10 young/adult sp0 : 20 / 108 sp1 : 84 / 432
10 [130, 550]
11 young/adult sp0 : 27 / 109 sp1 : 79 / 411
11 [130, 550]
12 young/adult sp0 : 25 / 120 sp1 : 81 / 431
12 [130, 550]
13 young/adult sp0 : 24 / 107 sp1 : 78 / 436
13 [130, 550]
14 young/adult sp0 : 26 / 114 sp1 : 80 / 426
14 [130, 550]
15 young/adult sp0 : 26 / 111 sp1 : 77 / 442
15 [130, 550]
16 young/adult sp0 : 27 / 115 sp1 : 78 / 434
16 [130, 550]
17 young/adult sp0

140 young/adult sp0 : 34 / 120 sp1 : 74 / 444
140 [150, 550]
141 young/adult sp0 : 27 / 125 sp1 : 81 / 440
141 [150, 550]
142 young/adult sp0 : 31 / 130 sp1 : 78 / 432
142 [150, 550]
143 young/adult sp0 : 28 / 133 sp1 : 83 / 426
143 [150, 550]
144 young/adult sp0 : 29 / 132 sp1 : 79 / 430
144 [150, 550]
145 young/adult sp0 : 31 / 121 sp1 : 76 / 444
145 [150, 550]
146 young/adult sp0 : 27 / 116 sp1 : 81 / 447
146 [150, 550]
147 young/adult sp0 : 28 / 126 sp1 : 82 / 433
147 [150, 550]
148 young/adult sp0 : 28 / 144 sp1 : 79 / 415
148 [150, 550]
149 young/adult sp0 : 31 / 121 sp1 : 76 / 441
149 [150, 550]
150 young/adult sp0 : 27 / 130 sp1 : 81 / 436
150 [150, 550]
151 young/adult sp0 : 26 / 133 sp1 : 82 / 421
151 [150, 550]
152 young/adult sp0 : 28 / 117 sp1 : 81 / 439
152 [150, 550]
153 young/adult sp0 : 30 / 131 sp1 : 76 / 430
153 [150, 550]
154 young/adult sp0 : 25 / 119 sp1 : 81 / 441
154 [150, 550]
155 young/adult sp0 : 30 / 125 sp1 : 79 / 437
155 [150, 550]
156 young/adult sp0 : 27

200

# Outputs:

Need Genepop file outputs for each generation (or every 5 years) after breeding to include juveniles. The effective population size estimates will be done separately.

Output info on age and sex of individuals would also help with Ne estimates.